In [ ]:
import socket
import time
from pop import PwmController, Led
from pop import Fan as PopFan

# ---------------------- FAN ----------------------
class Fan:
    def __init__(self):
        self.fan = PopFan(17)

    def on(self):
        self.fan.on()

    def off(self):
        self.fan.off()


# ---------------------- 일반 LED ----------------------
class LightLED:
    def __init__(self, led_pins):
        self.led_pins = led_pins
        self.leds = [Led(pin) for pin in led_pins]

    def on(self):
        for i, led in enumerate(self.leds):
            pin = self.led_pins[i]
            if pin != 27:
                led.on()

    def off(self):
        for led in self.leds:
            led.off()

# ---------------------- RGB 조명 ----------------------
class RGB:
    def __init__(self):
        self.pwm_r = PwmController()
        self.pwm_g = PwmController()
        self.pwm_b = PwmController()

        self.pwm_r.init()
        self.pwm_g.init()
        self.pwm_b.init()

        self.pwm_r.setChannel(0)
        self.pwm_g.setChannel(1)
        self.pwm_b.setChannel(2)

    def setRGB(self, r, g, b):
        self.pwm_r.setDuty(r)
        self.pwm_g.setDuty(g)
        self.pwm_b.setDuty(b)

    def warm(self):
        self.setRGB(60, 5, 0)

    def sleep(self):
        for d in range(0, 101, 5):
            self.setRGB(d, int(d * 0.4), 0)
            time.sleep(0.1)
    
    def rgb_off(self):
        self.setRGB(0, 0, 0)
        
    def rgb_on(self):
        self.setRGB(100, 100, 100)

# 윈도우 IP
server_ip = "192.168.101.102"
server_port = 39186

led = LightLED([23, 24, 25, 27])
rgb = RGB()
fan = Fan()

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((server_ip, server_port))
print("[PYTHON] Java 서버에 연결 완료")

try:
    while True:
        data = sock.recv(1024).decode().strip()
        if not data:
            print("[PYTHON] 서버 연결 종료")
            break

        print("[PYTHON] 서버로부터 명령 수신:", data)

        # ------- 여기서 LED/FAN/RGB 제어 --------
        if data == "LED_ON":
            led.on()
        elif data == "LED_OFF":
            led.off()
        elif data == "FAN_ON":
            fan.on()
        elif data == "FAN_OFF":
            fan.off()
        elif data == "LIGHT_WARM":
            rgb.warm()
        elif data == "LIGHT_SLEEP":
            rgb.sleep()
        elif data == "RGB_ON":
            rgb.rgb_on()
        elif data == "RGB_OFF":
            rgb.rgb_off()
        elif data.startswith("RGB_SET"):
            _, r, g, b = data.split()
            rgb.setRGB(int(r), int(g), int(b))


except Exception as e:
    print("에러:", e)

finally:
    sock.close()
